In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re
import scipy.stats as stats

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'
sys.path.append(pathPrefix + '/juno/work/taylorlab/friedman/myUtils')

import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util
import get_gene_and_cohort_list_utils

In [2]:
#given a maf of mutations in a case returns a set of unique position+case identifiers 
def adjust_gene_names_for_multiple_muts(caseMuts, mode='all'):
    
    nonsenseConsequences = set(['frameshift_variant', 'stop_gained'])
    caseMuts = caseMuts[caseMuts['Variant_Type'] != 'Intron']#get rid of intronic variants
    if mode == 'truncating':
        caseMuts = caseMuts[caseMuts['Consequence'].isin(nonsenseConsequences)]
    geneCounts = caseMuts['Hugo_Symbol'].value_counts()
    genesWithLotsOfMuts = set([])
    for gene, count in geneCounts.items():
        if count >= 2:
            genesWithLotsOfMuts.add(gene)
    ids = set(caseMuts[caseMuts['Hugo_Symbol'].isin(genesWithLotsOfMuts)]['casePosition'])
    return ids

In [ ]:
#
#
##TODO count double mutations in genes and oncogenic burden

In [8]:
mc3maf = pd.read_table('/Users/friedman/Desktop/stuffForNov25/mc3.v0.2.8.PUBLIC.LAML_PATCH_prepped_facets_oncokb.maf')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (5,139,140,141,144,146,147,149) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#USE THIS INSTEAD
#essentialGenes2 = get_gene_and_cohort_list_utils.get_essential_genes(depMapPath = pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/achillesDepMap.csv')
essentialGenes =set(['SCFD1', 'UBL5', 'NCBP1', 'HSPA5', 'BUB1B', 'NELFB', 'TIMELESS', 'HSPA9', 'SRSF11', 'MFAP1', 'PPWD1', 'PDCD11', 'SMG5', 'BYSL', 'MVK', 'SMG1', 'NCBP2', 'SUPT5H', 'PPP4C', 'SYS1', 'EBNA1BP2', 'GRWD1', 'MRPL57', 'ESPL1', 'NAPA', 'WEE1', 'TARDBP', 'METTL16', 'NUP205', 'DHX8', 'TAF1C', 'WDR5', 'OSGEP', 'WDR3', 'SRP14', 'NUP93', 'DDX49', 'TRAPPC8', 'TTI2', 'TTI1', 'CCND1', 'TRAPPC3', 'DDX42', 'TRAPPC4', 'DDX47', 'PSMB6', 'MTREX', 'SNRNP27', 'POP5', 'PPIL4', 'CTCF', 'NOP9', 'CAPZB', 'ATP6V1F', 'ATP6V1A', 'ATP2A2', 'DTL', 'GEMIN6', 'NOP2', 'TBCB', 'POLR1A', 'POLR1B', 'POLR1C', 'KRR1', 'PRIM1', 'XPO1', 'BMS1', 'CHCHD4', 'WDR12', 'CCT2', 'CCT3', 'PSMB4', 'CCT4', 'CCT5', 'EFTUD2', 'PAM16', 'WDR55', 'DMAP1', 'PSMG3', 'CDAN1', 'GGTLC2', 'ATP1A1', 'SNRNP200', 'SENP6', 'NCAPD3', 'PPP1R10', 'SMG6', 'HIST2H3A', 'SARS', 'BRF2', 'ANAPC2', 'INTS7', 'ANAPC4', 'SLC39A7', 'PDAP1', 'DNAJC9', 'PSMD14', 'PSMD11', 'PSMD13', 'TUBGCP5', 'PDRG1', 'UTP20', 'EMC3', 'EIF4A1', 'EIF4A3', 'TBCC', 'TIMM10', 'NUP133', 'PPP2R1A', 'LETM1', 'PELP1', 'ZNF830', 'POLE', 'CDC123', 'DCTN6', 'CDC6', 'DONSON', 'EXOSC10', 'BRIX1', 'NEDD1', 'SMC3', 'SMC2', 'PGS1', 'NOB1', 'CDC37', 'RBM17', 'POLR2K', 'PSMD4', 'BUD23', 'RBM19', 'DDB1', 'PSMD7', 'NUS1', 'TTC27', 'PGK1', 'SF3B5', 'NOL9', 'CYCS', 'ALG11', 'ALG13', 'ALG14', 'EIF1AX', 'RIOK2', 'RIOK1', 'ZNF738', 'CCDC115', 'ATP6V1E1', 'AK6', 'GEMIN5', 'SF3B4', 'BANF1', 'HSPE1', 'MARS', 'CDC45', 'RFT1', 'CMTR1', 'TUBGCP3', 'TUBGCP4', 'SPC25', 'CDC42', 'WDR7', 'PCID2', 'IPO11', 'RPL23A', 'EEF1A1', 'CHMP4B', 'PSMD6', 'PSMD1', 'RBBP5', 'PSMD3', 'PSMD2', 'SRBD1', 'CHAF1A', 'WDR82', 'CHAF1B', 'ERH', 'CPSF4', 'CPSF3', 'CPSF2', 'EIF2S3', 'PGD', 'CCNA2', 'PMPCB', 'DDX10', 'VPS25', 'EIF2S1', 'VPS28', 'DYNC1I2', 'PELO', 'CPSF1', 'CDCA5', 'CNOT3', 'CDCA8', 'DYNLRB1', 'PSMC5', 'PSMC6', 'RNMT', 'TONSL', 'ARIH1', 'HEATR1', 'MIS18BP1', 'KPNB1', 'KIF18A', 'RRN3', 'YARS2', 'SPATA5L1', 'ACTR6', 'RPS27A', 'THAP1', 'SNW1', 'PMPCA', 'LTO1', 'OGT', 'YRDC', 'KTI12', 'SNRNP25', 'RABGGTB', 'RABGGTA', 'RPP14', 'RBM28', 'SUPV3L1', 'WDR75', 'EWSR1', 'EEF2', 'RPL11', 'TRMT112', 'NUTF2', 'UBTF', 'AHCTF1', 'HUWE1', 'SF1', 'PES1', 'RPL23', 'NUDT21', 'SON', 'PRPF4', 'PRPF6', 'EIF3I', 'EIF3M', 'HYOU1', 'RAN', 'EIF3A', 'EIF3B', 'EIF3D', 'EIF3E', 'EIF3G', 'PCF11', 'UFD1', 'URB1', 'NOL10', 'DNAJC17', 'CENPN', 'CENPK', 'MRPL33', 'IMP4', 'SLC7A6OS', 'NPIPB6', 'GUK1', 'SNU13', 'CENPW', 'THOC2', 'THOC1', 'PAFAH1B1', 'ATP6V1B2', 'SDE2', 'PSMC3', 'PSMG4', 'TRAIP', 'RNPC3', 'DDX41', 'WNK1', 'ABCE1', 'NUP88', 'COPS8', 'RPL37A', 'SYF2', 'ALG2', 'COPS3', 'RPL27A', 'ALG1', 'COPS6', 'NUP85', 'COPS4', 'COPS5', 'EXOSC8', 'MED6', 'KIF23', 'MED4', 'EXOSC2', 'MED8', 'DHX33', 'EXOSC6', 'SPOUT1', 'EXOSC4', 'DHX37', 'COPZ1', 'GTF2E1', 'RUVBL1', 'RUVBL2', 'U2AF1', 'U2AF2', 'DDX56', 'DDX54', 'DDX55', 'SF3A1', 'SF3A2', 'SF3A3', 'NOL6', 'VPS41', 'PKM', 'INCENP', 'DHPS', 'RNPS1', 'BUB3', 'RPP21', 'MMS22L', 'NCAPG', 'BOP1', 'EIF4E', 'ZBTB11', 'PLA2G10', 'MCM6', 'FCF1', 'IK', 'DTYMK', 'RSL24D1', 'SASS6', 'MDN1', 'HDAC3', 'PI4KA', 'RFC5', 'RPLP2', 'RPLP1', 'RPLP0', 'EIF2B2', 'EIF2B3', 'RFC3', 'RFC2', 'ELOB', 'KIF11', 'PPA1', 'ADSL', 'WDHD1', 'NDC80', 'RNF113A', 'INTS11', 'ECD', 'HINFP', 'RACGAP1', 'CDT1', 'SNRPA1', 'ETF1', 'POLD3', 'TRAPPC11', 'UTP15', 'NAE1', 'SRP72', 'SMC1A', 'GPN2', 'ATP6V0B', 'ATP6V0C', 'DPAGT1', 'RRS1', 'CARS', 'U2SURP', 'YKT6', 'MTOR', 'RPS4X', 'TPR', 'DUT', 'NAT10', 'NDOR1', 'TANGO6', 'PPAN', 'DCTN5', 'CDK1', 'TCP1', 'RPL10A', 'ISY1', 'CDK7', 'CDK9', 'XAB2', 'NRF1', 'MAD2L2', 'POLA2', 'PHB2', 'RPA1', 'SF3B1', 'USP5', 'SF3B3', 'SF3B2', 'ZMAT2', 'RRP12', 'GTF2A2', 'TUBB', 'CNIH4', 'ELAC2', 'NOP56', 'UBE2I', 'HSPD1', 'ABCB7', 'AURKA', 'AURKB', 'MED17', 'EPRS', 'XRCC6', 'INTS3', 'INTS1', 'COPA', 'LSM8', 'KRI1', 'CFAP298', 'INTS4', 'SBDS', 'LSM6', 'LSM7', 'SRSF7', 'LSM2', 'LSM3', 'NLE1', 'RPL7A', 'NBPF12', 'DYNC1H1', 'SRSF1', 'DDX23', 'DPH3', 'PMF1', 'DDX24', 'GTF2B', 'CSTF1', 'WDR74', 'TUBG1', 'SAE1', 'MED22', 'WDR77', 'TWISTNB', 'TARS', 'CKAP5', 'TBL3', 'TOMM40', 'RPN1', 'RAD9A', 'SYMPK', 'POLR3F', 'VCP', 'POLR3B', 'UQCRH', 'GGPS1', 'RPS9', 'CWC22', 'POLR3K', 'POLR3H', 'GPS1', 'URI1', 'PPP1R2', 'PRPF31', 'ATP6AP1', 'PPP1R8', 'ATP6AP2', 'PLK1', 'YARS', 'SACM1L', 'TUT1', 'RACK1', 'FAM25G', 'CUL1', 'LUC7L3', 'RPAIN', 'IARS', 'OIP5', 'PRMT1', 'PRMT5', 'RPL14', 'RPL15', 'RPL17', 'RPL10', 'CHERP', 'RPL12', 'RPL13', 'BRD4', 'RPL18', 'RPL19', 'USP36', 'KAT8', 'RBBP8', 'SNRPD3', 'SNRPD2', 'SNRPD1', 'PRPF8', 'RBM48', 'PHB', 'RBBP6', 'ARCN1', 'NKAP', 'CHMP2A', 'KIN', 'RTCB', 'COPB1', 'COPB2', 'MRPL21', 'SEH1L', 'DUX4', 'LYRM4', 'GAPDH', 'MED26', 'CIAO1', 'NOP10', 'CIAO3', 'NOP14', 'NOP16', 'SLU7', 'TIMM8A', 'PRPF3', 'TICRR', 'KANSL3', 'IARS2', 'CDC16', 'SRSF3', 'SRSF2', 'NARS', 'TRIAP1', 'TFRC', 'RPS3A', 'RBM39', 'CHEK1', 'GSPT1', 'ROMO1', 'CLNS1A', 'RAD51', 'GNL2', 'SEC13', 'NSMCE3', 'CDK11A', 'TRRAP', 'SFPQ', 'RSL1D1', 'UBA2', 'UBA3', 'CSE1L', 'PWP2', 'GINS2', 'GINS3', 'GINS1', 'UBE2M', 'GINS4', 'UBE2N', 'ZPR1', 'PSMB2', 'SHQ1', 'POLE2', 'ATP6V1G1', 'PHF5A', 'NVL', 'TOP3A', 'TAF8', 'SMN2', 'MIS18A', 'RPS12', 'ANAPC11', 'RPP30', 'UBA1', 'RPL18A', 'RPS15A', 'CTDP1', 'ANAPC1', 'UTP4', 'RPS3', 'FBXO5', 'UPF2', 'TOMM22', 'UTP3', 'HARS', 'CHMP6', 'CHMP5', 'RPS8', 'BRF1', 'RBM8A', 'MASTL', 'SS18L2', 'GARS', 'PSMA2', 'PSMA3', 'PSMA1', 'PSMA6', 'PSMA7', 'PSMA4', 'PSMA5', 'HMGCS1', 'IGBP1', 'RPS18', 'RPS23', 'RPS20', 'FOXD4', 'RPS28', 'RPS29', 'NAA15', 'HTATSF1', 'QARS', 'DBR1', 'LRR1', 'TXNL4A', 'NAA10', 'FARSB', 'YJU2', 'ZMAT5', 'WDR18', 'TRNT1', 'CSNK2B', 'TERF2', 'EIF3CL', 'VARS', 'IMP3', 'BIRC5', 'EIF5', 'EIF6', 'PRELID1', 'NIP7', 'NOC4L', 'RPAP1', 'DAD1', 'PTPN23', 'CHORDC1', 'ECT2', 'RPL24', 'RPL27', 'RPL26', 'NSF', 'FAU', 'COPG1', 'PCNA', 'ZNHIT2', 'SMARCB1', 'SUPT6H', 'MCM7', 'GTPBP4', 'MCM5', 'MCM4', 'MCM2', 'ALDOA', 'NOP58', 'NCAPH2', 'ANKLE2', 'NAA50', 'NOC3L', 'RPA3', 'RPA2', 'TSR1', 'NRDE2', 'SEC61A1', 'ILF3', 'ILF2', 'DHDDS', 'MTBP', 'DARS', 'PLK4', 'PKMYT1', 'THOC3', 'PUF60', 'AFG3L2', 'DHX15', 'BORA', 'PSMC2', 'NHLRC2', 'TPX2', 'MED11', 'THOC7', 'CIAPIN1', 'TRAPPC1', 'EIF1AD', 'SNRNP35', 'SAP18', 'RPE', 'MYC', 'UBR4', 'ISCU', 'RPP40', 'DDX3X', 'FNTB', 'NUF2', 'COG3', 'RANGAP1', 'SNAPC3', 'SNAPC2', 'SNAPC1', 'LSG1', 'RAE1', 'SNAPC5', 'POLR3A', 'CRCP', 'WDR43', 'TMA16', 'RPS6', 'PPP2CA', 'EEF1G', 'CCDC84', 'CDC7', 'GRB2', 'XRN2', 'UBC', 'ELP6', 'ELP5', 'CCNH', 'CINP', 'RPS2', 'NCAPG2', 'RPL8', 'RPL9', 'RPL6', 'RPL7', 'RPL4', 'RPL5', 'CDC5L', 'TUBGCP2', 'LONP1', 'FEN1', 'SSU72', 'CLTC', 'DNM2', 'MAT2A', 'MMS19', 'NFS1', 'SNRPE', 'NMD3', 'INTS5', 'OR4F4', 'HAUS6', 'HAUS7', 'HAUS4', 'HAUS5', 'HAUS1', 'SUPT16H', 'PCBP2', 'PPP1R12A', 'PCBP1', 'PRELID3B', 'CDIPT', 'INTS9', 'TOP2A', 'MRPL10', 'NEDD8', 'NACA', 'NXF1', 'MED30', 'WARS', 'POLR2E', 'POLR2D', 'POLR2G', 'POLR2F', 'PAK1IP1', 'POLR2C', 'POLR2B', 'SART3', 'POLR2L', 'SART1', 'POLR2I', 'POLR2H', 'HIST1H2BJ', 'POLR2J', 'TAF7', 'TAF6', 'TAF5', 'CDC27', 'ELP3', 'ANKRD20A1', 'CDC23', 'CDC20', 'GTF3A', 'UPF1', 'RBM25', 'RBM22', 'ELP1', 'ACTR8', 'BUD31', 'IPO7', 'SPC24', 'ACTR2', 'NRBP1', 'GRPEL1', 'CIAO2B', 'TBC1D3B', 'SMC4', 'HNRNPH1', 'SNRPC', 'NPLOC4', 'GPKOW', 'PAXBP1', 'SOD1', 'TNPO3', 'PRC1', 'GNL3L', 'HNRNPU', 'MARS2', 'POLD1', 'HNRNPK', 'GPN3', 'HNRNPM', 'GPN1', 'SAP30BP', 'SNRNP70', 'PDCD2', 'TAMM41', 'NIFK', 'PFDN2', 'ELL', 'PFDN6', 'NSA2', 'RBM14', 'RPL3', 'MNAT1', 'CRNKL1', 'ZNF525', 'SMU1', 'RPL36', 'RPL37', 'EP400', 'PSMB5', 'MRTO4', 'PSMB3', 'MAK16', 'FBL', 'RPS13', 'KARS', 'RPS11', 'RPL35', 'RPS17', 'RPS16', 'RPS15', 'RPS14', 'SSRP1', 'C1orf109', 'RPS19', 'ARL2', 'RPS24', 'VHL', 'RPS25', 'RRM2', 'RRM1', 'CACTIN', 'RNGTT', 'TXN', 'ZRSR2', 'SRP54', 'HCFC1', 'AQR', 'PRPF19', 'TSR2', 'NUP43', 'COPE', 'VPS13D', 'EXOSC5', 'PABPN1', 'ESF1', 'NEPRO', 'RAD21', 'ORC6', 'TINF2', 'WBP11', 'SNRPB', 'ARMC7', 'TBCD', 'SNRPF', 'SNRPG', 'CHD4', 'SCAP', 'RCC1', 'SBNO1', 'USPL1', 'RPL38', 'CLP1', 'PRPF38A', 'PRPF38B', 'AARS', 'RPL32', 'RPL31', 'ACTL6A', 'DKC1', 'RGPD6', 'SPATA5'])

In [10]:
tsgs = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])
impactGenes = set(['ABL1', 'ACVR1', 'AGO2', 'AKT1', 'AKT2', 'AKT3', 'ALK', 'ALOX12B', 'ANKRD11', 'APC', 'AR', 'ARAF', 'ARID1A', 'ARID1B', 'ARID2', 'ARID5B', 'ASXL1', 'ASXL2', 'ATM', 'ATR', 'ATRX', 'AURKA', 'AURKB', 'AXIN1', 'AXIN2', 'AXL', 'B2M', 'BABAM1', 'BAP1', 'BARD1', 'BBC3', 'BCL10', 'BCL2', 'BCL2L1', 'BCL2L11', 'BCL6', 'BCOR', 'BIRC3', 'BLM', 'BMPR1A', 'BRAF', 'BRCA1', 'BRCA2', 'BRD4', 'BRIP1', 'BTK', 'CALR', 'CARD11', 'CARM1', 'CASP8', 'CBFB', 'CBL', 'CCND1', 'CCND2', 'CCND3', 'CCNE1', 'CD274', 'CD276', 'CD79A', 'CD79B', 'CDC42', 'CDC73', 'CDH1', 'CDK12', 'CDK4', 'CDK6', 'CDK8', 'CDKN1A', 'CDKN1B', 'CDKN2A', 'CDKN2B', 'CDKN2C', 'CEBPA', 'CENPA', 'CHEK1', 'CHEK2', 'CIC', 'CREBBP', 'CRKL', 'CRLF2', 'CSDE1', 'CSF1R', 'CSF3R', 'CTCF', 'CTLA4', 'CTNNB1', 'CUL3', 'CXCR4', 'CYLD', 'CYSLTR2', 'DAXX', 'DCUN1D1', 'DDR2', 'DICER1', 'DIS3', 'DNAJB1', 'DNMT1', 'DNMT3A', 'DNMT3B', 'DOT1L', 'DROSHA', 'DUSP4', 'E2F3', 'EED', 'EGFL7', 'EGFR', 'EIF1AX', 'EIF4A2', 'EIF4E', 'ELF3', 'EP300', 'EPAS1', 'EPCAM', 'EPHA3', 'EPHA5', 'EPHA7', 'EPHB1', 'ERBB2', 'ERBB3', 'ERBB4', 'ERCC2', 'ERCC3', 'ERCC4', 'ERCC5', 'ERF', 'ERG', 'ERRFI1', 'ESR1', 'ETV1', 'ETV6', 'EZH1', 'EZH2', 'FAM123B', 'FAM175A', 'FAM46C', 'FAM58A', 'FANCA', 'FANCC', 'FAT1', 'FBXW7', 'FGF19', 'FGF3', 'FGF4', 'FGFR1', 'FGFR2', 'FGFR3', 'FGFR4', 'FH', 'FLCN', 'FLT1', 'FLT3', 'FLT4', 'FOXA1', 'FOXL2', 'FOXO1', 'FOXP1', 'FUBP1', 'FYN', 'GATA1', 'GATA2', 'GATA3', 'GLI1', 'GNA11', 'GNAQ', 'GNAS', 'GPS2', 'GREM1', 'GRIN2A', 'GSK3B', 'H3F3A', 'H3F3B', 'H3F3C', 'HGF', 'HIST1H1C', 'HIST1H2BD', 'HIST1H3A', 'HIST1H3B', 'HIST1H3C', 'HIST1H3D', 'HIST1H3E', 'HIST1H3F', 'HIST1H3G', 'HIST1H3H', 'HIST1H3I', 'HIST1H3J', 'HIST2H3C', 'HIST2H3D', 'HIST3H3', 'HLA-A', 'HLA-B', 'HNF1A', 'HOXB13', 'HRAS', 'ICOSLG', 'ID3', 'IDH1', 'IDH2', 'IFNGR1', 'IGF1', 'IGF1R', 'IGF2', 'IKBKE', 'IKZF1', 'IL10', 'IL7R', 'INHA', 'INHBA', 'INPP4A', 'INPP4B', 'INPPL1', 'INSR', 'IRF4', 'IRS1', 'IRS2', 'JAK1', 'JAK2', 'JAK3', 'JUN', 'KDM5A', 'KDM5C', 'KDM6A', 'KDR', 'KEAP1', 'KIT', 'KLF4', 'KMT2B', 'KMT5A', 'KNSTRN', 'KRAS', 'LATS1', 'LATS2', 'LMO1', 'LYN', 'MALT1', 'MAP2K1', 'MAP2K2', 'MAP2K4', 'MAP3K1', 'MAP3K13', 'MAP3K14', 'MAPK1', 'MAPK3', 'MAPKAP1', 'MAX', 'MCL1', 'MDC1', 'MDM2', 'MDM4', 'MED12', 'MEF2B', 'MEN1', 'MET', 'MGA', 'MITF', 'MLH1', 'KMT2A', 'KMT2B', 'KMT2C', 'MPL', 'MRE11A', 'MSH2', 'MSH3', 'MSH6', 'MSI1', 'MSI2', 'MST1', 'MST1R', 'MTOR', 'MUTYH', 'MYC', 'MYCL1', 'MYCN', 'MYD88', 'MYOD1', 'NBN', 'NCOA3', 'NCOR1', 'NEGR1', 'NF1', 'NF2', 'NFE2L2', 'NFKBIA', 'NKX2-1', 'NKX3-1', 'NOTCH1', 'NOTCH2', 'NOTCH3', 'NOTCH4', 'NPM1', 'NRAS', 'NSD1', 'NTHL1', 'NTRK1', 'NTRK2', 'NTRK3', 'NUF2', 'NUP93', 'PAK1', 'PAK7', 'PALB2', 'PARK2', 'PARP1', 'PAX5', 'PBRM1', 'PDCD1', 'PDCD1LG2', 'PDGFRA', 'PDGFRB', 'PDPK1', 'PGR', 'PHOX2B', 'PIK3C2G', 'PIK3C3', 'PIK3CA', 'PIK3CB', 'PIK3CD', 'PIK3CG', 'PIK3R1', 'PIK3R2', 'PIK3R3', 'PIM1', 'PLCG2', 'PLK2', 'PMAIP1', 'PMS1', 'PMS2', 'PNRC1', 'POLD1', 'POLE', 'PPARG', 'PPM1D', 'PPP2R1A', 'PPP4R2', 'PPP6C', 'PRDM1', 'PRDM14', 'PREX2', 'PRKAR1A', 'PRKCI', 'PRKD1', 'PTCH1', 'PTEN', 'PTP4A1', 'PTPN11', 'PTPRD', 'PTPRS', 'PTPRT', 'RAB35', 'RAC1', 'RAC2', 'RAD21', 'RAD50', 'RAD51', 'RAD51C', 'RAD51L1', 'RAD51L3', 'RAD52', 'RAD54L', 'RAF1', 'RARA', 'RASA1', 'RB1', 'RBM10', 'RECQL', 'RECQL4', 'REL', 'RET', 'RFWD2', 'RHEB', 'RHOA', 'RICTOR', 'RIT1', 'RNF43', 'ROS1', 'RPS6KA4', 'RPS6KB2', 'RPTOR', 'RRAGC', 'RRAS', 'RRAS2', 'RTEL1', 'RUNX1', 'RXRA', 'RYBP', 'SDHA', 'SDHAF2', 'SDHB', 'SDHC', 'SDHD', 'SESN1', 'SESN2', 'SESN3', 'SETD2', 'SF3B1', 'SH2B3', 'SH2D1A', 'SHOC2', 'SHQ1', 'SLX4', 'SMAD2', 'SMAD3', 'SMAD4', 'SMARCA4', 'SMARCB1', 'SMARCD1', 'SMO', 'SMYD3', 'SOCS1', 'SOS1', 'SOX17', 'SOX2', 'SOX9', 'SPEN', 'SPOP', 'SPRED1', 'SRC', 'SRSF2', 'STAG2', 'STAT3', 'STAT5A', 'STAT5B', 'STK11', 'STK19', 'STK40', 'SUFU', 'SUZ12', 'SYK', 'TAP1', 'TAP2', 'TBX3', 'TCEB1', 'TCF3', 'TCF7L2', 'TEK', 'TERT', 'TET1', 'TET2', 'TGFBR1', 'TGFBR2', 'TMEM127', 'TMPRSS2', 'TNFAIP3', 'TNFRSF14', 'TOP1', 'TP53', 'TP53BP1', 'TP63', 'TRAF2', 'TRAF7', 'TSC1', 'TSC2', 'TSHR', 'U2AF1', 'UPF1', 'VEGFA', 'VHL', 'VTCN1', 'WHSC1', 'WHSC1L1', 'WT1', 'WWTR1', 'XIAP', 'XPO1', 'XRCC2', 'YAP1', 'YES1', 'ZFHX3', 'ZRSR2'])
impactAndEssentialGenes = impactGenes | essentialGenes


In [72]:
#tmb info
tmbDict = dict(mc3maf['SAMPLE_ID'].value_counts()/30.0)
lowTMBCases = set([key for key, tmb in tmbDict.items() if tmb < 3])

In [93]:
#mmrCases, poleCases = get_gene_and_cohort_list_utils.get_tcga_pole_mmr_hypermutator_ids(tcgaSigsPath = pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/tcgaSigsCombined.txt')
mmrCases = set(['TCGA-BR-8361-01', 'TCGA-D7-A4YV-01', 'TCGA-AA-3984-01', 'TCGA-D5-6530-01', 'TCGA-BR-6452-01', 'TCGA-VS-A94Z-01', 'TCGA-F1-6874-01', 'TCGA-CG-4306-01', 'TCGA-A6-2686-01', 'TCGA-AA-3492-01', 'TCGA-AJ-A3OL-01', 'TCGA-AX-A1C9-01', 'TCGA-HU-A4G8-01', 'TCGA-EJ-7782-01', 'TCGA-AP-A0LS-01', 'TCGA-HF-A5NB-01', 'TCGA-EC-A24G-01', 'TCGA-KN-8428-01', 'TCGA-EE-A29L-06', 'TCGA-AA-3713-01', 'TCGA-BR-4370-01', 'TCGA-D1-A177-01', 'TCGA-D1-A167-01', 'TCGA-L5-A43J-01', 'TCGA-B0-5098-01', 'TCGA-AD-6895-01', 'TCGA-CN-5369-01', 'TCGA-B5-A11G-01', 'TCGA-A6-6653-01', 'TCGA-EY-A1GK-01', 'TCGA-CM-4746-01', 'TCGA-DF-A2KZ-01', 'TCGA-SJ-A6ZI-01', 'TCGA-EA-A3HS-01', 'TCGA-HF-7132-01', 'TCGA-A5-A2K3-01', 'TCGA-D1-A17Q-01', 'TCGA-BR-4368-01', 'TCGA-D3-A8GM-06', 'TCGA-EE-A3AB-06', 'TCGA-BS-A0TA-01', 'TCGA-CD-A4MG-01', 'TCGA-EY-A5W2-01', 'TCGA-CK-5916-01', 'TCGA-BR-4201-01', 'TCGA-BR-7703-01', 'TCGA-D1-A101-01', 'TCGA-G4-6628-01', 'TCGA-G4-6309-01', 'TCGA-B5-A0K9-01', 'TCGA-AP-A1E1-01', 'TCGA-AO-A128-01', 'TCGA-QG-A5Z2-01', 'TCGA-BS-A0V4-01', 'TCGA-EW-A2FV-01', 'TCGA-VQ-AA6D-01', 'TCGA-D5-6540-01', 'TCGA-A5-A0VQ-01', 'TCGA-HU-8602-01', 'TCGA-D1-A1O7-01', 'TCGA-KK-A59V-01', 'TCGA-BR-4292-01', 'TCGA-MA-AA42-01', 'TCGA-A5-A2K7-01', 'TCGA-AD-6889-01', 'TCGA-AD-5900-01', 'TCGA-EE-A2GI-06', 'TCGA-BG-A0VW-01', 'TCGA-B5-A3FC-01', 'TCGA-AY-6197-01', 'TCGA-FI-A2F4-01', 'TCGA-EE-A3AA-06', 'TCGA-CD-8536-01', 'TCGA-D7-A4YY-01', 'TCGA-AD-A5EJ-01', 'TCGA-HU-A4H8-01', 'TCGA-BS-A0UV-01', 'TCGA-EY-A548-01', 'TCGA-AX-A2HJ-01', 'TCGA-NH-A5IV-01', 'TCGA-CM-5861-01', 'TCGA-AP-A1DO-01', 'TCGA-BG-A222-01', 'TCGA-D8-A1XK-01', 'TCGA-F7-A624-01', 'TCGA-AM-5821-01', 'TCGA-AA-A01R-01', 'TCGA-F4-6570-01', 'TCGA-CM-6171-01', 'TCGA-HU-A4H3-01', 'TCGA-G4-6588-01', 'TCGA-EY-A215-01', 'TCGA-CK-6746-01', 'TCGA-AX-A3FS-01', 'TCGA-E6-A2P8-01', 'TCGA-B7-5816-01', 'TCGA-EY-A1GF-01', 'TCGA-EE-A181-06', 'TCGA-BG-A18B-01', 'TCGA-BR-8363-01', 'TCGA-EO-A3KX-01', 'TCGA-BS-A0VI-01', 'TCGA-BS-A0UA-01', 'TCGA-EY-A2OM-01', 'TCGA-B5-A11E-01', 'TCGA-BG-A3EW-01', 'TCGA-VQ-A8E3-01', 'TCGA-BR-8368-01', 'TCGA-EY-A1GU-01', 'TCGA-AJ-A8CW-01', 'TCGA-F1-6177-01', 'TCGA-D5-6927-01', 'TCGA-EO-A22T-01', 'TCGA-AJ-A23O-01', 'TCGA-G4-6304-01', 'TCGA-BG-A2L7-01', 'TCGA-HU-A4H4-01', 'TCGA-MX-A5UJ-01', 'TCGA-VQ-A8PP-01', 'TCGA-A6-5665-01', 'TCGA-EY-A1GQ-01', 'TCGA-EC-A1NJ-01', 'TCGA-DI-A0WH-01', 'TCGA-D1-A103-01', 'TCGA-BR-8372-01', 'TCGA-BR-6566-01', 'TCGA-BR-8059-01', 'TCGA-D1-A1NS-01', 'TCGA-G4-6586-01', 'TCGA-4R-AA8I-01', 'TCGA-AP-A1DR-01', 'TCGA-D8-A1XQ-01', 'TCGA-B5-A11J-01', 'TCGA-BS-A0UJ-01', 'TCGA-EC-A1QX-01', 'TCGA-CG-4442-01', 'TCGA-AX-A1C5-01', 'TCGA-AP-A0LP-01', 'TCGA-B5-A5OC-01', 'TCGA-HU-A4G9-01', 'TCGA-CG-5721-01', 'TCGA-AX-A2HG-01', 'TCGA-VQ-A8P2-01', 'TCGA-BS-A0UL-01', 'TCGA-FI-A2D6-01', 'TCGA-95-7567-01', 'TCGA-AX-A3G9-01', 'TCGA-EA-A410-01', 'TCGA-AA-3510-01', 'TCGA-CV-A6K1-01', 'TCGA-63-A5MM-01', 'TCGA-B5-A1MX-01', 'TCGA-B5-A11Y-01', 'TCGA-BR-8487-01', 'TCGA-AX-A1C4-01', 'TCGA-D1-A17U-01', 'TCGA-D1-A17R-01', 'TCGA-HU-A4GN-01', 'TCGA-94-8490-01', 'TCGA-AA-A010-01', 'TCGA-EE-A29E-06', 'TCGA-F4-6856-01', 'TCGA-D1-A2G0-01', 'TCGA-EI-6882-01', 'TCGA-BR-8081-01', 'TCGA-CD-A4MJ-01', 'TCGA-EI-6507-01', 'TCGA-BR-4256-01', 'TCGA-AZ-4315-01', 'TCGA-CM-6674-01', 'TCGA-CM-4743-01', 'TCGA-PG-A917-01', 'TCGA-AJ-A2QO-01', 'TCGA-DF-A2KN-01', 'TCGA-N7-A4Y0-01', 'TCGA-CA-6717-01', 'TCGA-D5-6930-01', 'TCGA-IB-7651-01', 'TCGA-AA-A01P-01', 'TCGA-F4-6703-01', 'TCGA-EO-A22R-01', 'TCGA-DM-A1HB-01', 'TCGA-AX-A1CE-01', 'TCGA-A5-A0G1-01', 'TCGA-EO-A22S-01', 'TCGA-BR-7851-01', 'TCGA-AA-3663-01', 'TCGA-D1-A0ZS-01', 'TCGA-B5-A0JY-01', 'TCGA-2W-A8YY-01', 'TCGA-EO-A3AZ-01', 'TCGA-B5-A11H-01', 'TCGA-AX-A2HD-01', 'TCGA-D1-A17D-01', 'TCGA-BR-7707-01', 'TCGA-BR-8680-01', 'TCGA-PK-A5HB-01', 'TCGA-AX-A0J0-01', 'TCGA-AX-A3FT-01', 'TCGA-AJ-A3BH-01', 'TCGA-VS-A9U6-01', 'TCGA-BR-4362-01', 'TCGA-A6-5661-01', 'TCGA-SL-A6JA-01', 'TCGA-D1-A1NY-01', 'TCGA-CK-4951-01', 'TCGA-L5-A4OI-01', 'TCGA-QS-A5YR-01', 'TCGA-BR-8078-01', 'TCGA-EY-A1H0-01', 'TCGA-QK-A6VB-01', 'TCGA-AA-3947-01', 'TCGA-B7-A5TI-01', 'TCGA-VS-A9UJ-01', 'TCGA-VQ-A91K-01', 'TCGA-AP-A1DV-01', 'TCGA-AP-A059-01', 'TCGA-EO-A3AU-01', 'TCGA-B5-A11U-01', 'TCGA-BR-4363-01', 'TCGA-F5-6814-01', 'TCGA-G4-6302-01', 'TCGA-BS-A0TJ-01', 'TCGA-ZF-AA4W-01', 'TCGA-FI-A2D4-01', 'TCGA-DU-6392-01', 'TCGA-BS-A0TE-01', 'TCGA-AP-A1DK-01', 'TCGA-D8-A1J8-01', 'TCGA-EO-A22U-01', 'TCGA-VQ-A8PB-01', 'TCGA-VQ-A8PX-01', 'TCGA-A5-A0VP-01', 'TCGA-D1-A15X-01', 'TCGA-EE-A2MD-06', 'TCGA-AP-A0LM-01', 'TCGA-BK-A13B-01', 'TCGA-CG-5723-01', 'TCGA-FI-A2D0-01', 'TCGA-BR-6802-01', 'TCGA-CG-5726-01', 'TCGA-CG-4465-01', 'TCGA-DK-A6AW-01', 'TCGA-AJ-A3BG-01', 'TCGA-D5-6928-01', 'TCGA-EW-A1IZ-01', 'TCGA-CD-A4MI-01', 'TCGA-BR-8382-01', 'TCGA-D1-A174-01', 'TCGA-AJ-A3EK-01', 'TCGA-D1-A17M-01', 'TCGA-WQ-A9G7-01', 'TCGA-D1-A0ZO-01', 'TCGA-BR-6852-01', 'TCGA-AP-A1DM-01', 'TCGA-EI-6917-01', 'TCGA-D1-A17H-01', 'TCGA-SW-A7EA-01', 'TCGA-BH-A18G-01', 'TCGA-A6-A565-01', 'TCGA-D1-A1NZ-01', 'TCGA-AU-6004-01', 'TCGA-VQ-A924-01', 'TCGA-DI-A1BU-01', 'TCGA-CK-5913-01', 'TCGA-R5-A7ZI-01', 'TCGA-BR-4257-01', 'TCGA-AP-A0LE-01', 'TCGA-06-5416-01', 'TCGA-D1-A160-01', 'TCGA-E6-A1LX-01', 'TCGA-AZ-4615-01', 'TCGA-FI-A2D5-01', 'TCGA-EE-A2GO-06', 'TCGA-AZ-6598-01', 'TCGA-B5-A0JU-01', 'TCGA-AJ-A3EL-01', 'TCGA-WS-AB45-01', 'TCGA-A5-A1OF-01', 'TCGA-AX-A06H-01', 'TCGA-BS-A0U8-01', 'TCGA-55-8506-01', 'TCGA-AP-A1DP-01', 'TCGA-F1-A448-01', 'TCGA-HU-A4GX-01', 'TCGA-AX-A3G8-01', 'TCGA-BR-8591-01', 'TCGA-BG-A0MQ-01', 'TCGA-B5-A1MW-01', 'TCGA-BR-4280-01', 'TCGA-AX-A2IN-01', 'TCGA-AD-6964-01', 'TCGA-D1-A163-01', 'TCGA-D7-A6EY-01', 'TCGA-VQ-A91D-01', 'TCGA-E6-A2P9-01', 'TCGA-QQ-A8VF-01', 'TCGA-HU-A4GU-01', 'TCGA-EO-A22X-01', 'TCGA-PG-A6IB-01', 'TCGA-ZB-A966-01', 'TCGA-CG-4305-01', 'TCGA-RD-A8NB-01', 'TCGA-BH-A0HA-01', 'TCGA-86-A4JF-01', 'TCGA-B5-A3FA-01', 'TCGA-XK-AAIW-01', 'TCGA-DF-A2KU-01', 'TCGA-VQ-A8PO-01', 'TCGA-BR-4361-01', 'TCGA-AP-A1DH-01', 'TCGA-EY-A2OP-01', 'TCGA-AJ-A3OJ-01', 'TCGA-G4-6320-01', 'TCGA-5M-AAT6-01', 'TCGA-BG-A0VZ-01', 'TCGA-HU-A4GT-01', 'TCGA-ND-A4WC-01', 'TCGA-D1-A17A-01', 'TCGA-3B-A9HT-01', 'TCGA-EY-A1GO-01', 'TCGA-EO-A3B0-01', 'TCGA-HU-A4GQ-01', 'TCGA-L5-A8NM-01', 'TCGA-D1-A176-01', 'TCGA-AX-A2HA-01'])
poleCases = set(['TCGA-DU-6392-01', 'TCGA-AA-3984-01', 'TCGA-AJ-A3EK-01', 'TCGA-B5-A0JY-01', 'TCGA-AX-A2HD-01', 'TCGA-AP-A1E0-01', 'TCGA-AZ-4315-01', 'TCGA-EO-A3AV-01', 'TCGA-DF-A2KU-01', 'TCGA-E6-A1M0-01', 'TCGA-EI-6917-01', 'TCGA-DF-A2KN-01', 'TCGA-AX-A2HC-01', 'TCGA-EO-A3AY-01', 'TCGA-QF-A5YS-01', 'TCGA-B5-A11N-01', 'TCGA-EO-A3KX-01', 'TCGA-B5-A11E-01', 'TCGA-AX-A1CE-01', 'TCGA-BS-A0UF-01', 'TCGA-ND-A4WC-01', 'TCGA-06-5416-01', 'TCGA-EO-A22R-01', 'TCGA-2W-A8YY-01', 'TCGA-FI-A2D5-01', 'TCGA-AJ-A3EL-01', 'TCGA-B5-A3FA-01', 'TCGA-AJ-A5DW-01', 'TCGA-CA-6717-01', 'TCGA-XK-AAIW-01', 'TCGA-BR-8680-01', 'TCGA-AX-A0J0-01', 'TCGA-D1-A103-01', 'TCGA-AJ-A3BH-01', 'TCGA-BK-A6W3-01', 'TCGA-D1-A16X-01', 'TCGA-IB-7651-01', 'TCGA-E6-A1LX-01', 'TCGA-A5-A1OF-01', 'TCGA-BS-A0TC-01', 'TCGA-EW-A2FV-01', 'TCGA-EO-A22X-01', 'TCGA-D1-A17Q-01', 'TCGA-AP-A1DV-01', 'TCGA-AP-A059-01', 'TCGA-A5-A0G1-01', 'TCGA-VQ-A8P2-01', 'TCGA-EO-A3B0-01', 'TCGA-D1-A16Y-01', 'TCGA-F5-6814-01', 'TCGA-EY-A1GI-01', 'TCGA-AA-3510-01', 'TCGA-B5-A3FC-01', 'TCGA-AP-A1DK-01', 'TCGA-DK-A6AW-01', 'TCGA-EO-A22U-01', 'TCGA-QS-A5YQ-01', 'TCGA-B5-A1MR-01', 'TCGA-A5-A2K5-01', 'TCGA-BS-A0UV-01', 'TCGA-AP-A0LM-01', 'TCGA-DF-A2KV-01', 'TCGA-AA-A010-01', 'TCGA-EY-A215-01', 'TCGA-B5-A1MX-01', 'TCGA-CA-6718-01', 'TCGA-EY-A1GD-01'])
minArbitraryTmbThresh = 25
mmrCasesReduced = set([])
for mid in mmrCases:
    if mid in tmbDict and tmbDict[mid] > minArbitraryTmbThresh:
        mmrCasesReduced.add(mid)

In [98]:
analyzeIds = mmrCases | poleCases | lowTMBCases
analyzeMaf = mc3maf[mc3maf['SAMPLE_ID'].isin(analyzeIds)]
#print 'setting up case position'
analyzeMaf['casePosition'] = analyzeMaf.apply(lambda row: str(row['Start_Position']) +'_' + row['SAMPLE_ID'], axis=1)

cases = set(analyzeMaf['Tumor_Sample_Barcode'])
doubleIds = set([])
cntr = 0
cntr += 1
for case in cases:
    if cntr%50==0: print cntr
    cntr += 1
    caseMaf = analyzeMaf[analyzeMaf['Tumor_Sample_Barcode'] == case]
    caseIds = adjust_gene_names_for_multiple_muts(caseMaf, mode='all')
    doubleIds = doubleIds | caseIds
analyzeMaf['isDouble'] = analyzeMaf['casePosition'].apply(lambda x: True if x in doubleIds else False)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [75]:
#ONLY RUN THIS CELL FOR SPECIFIC COMPARISSONs

listOfDicts = []
for case in set(analyzeMaf['Tumor_Sample_Barcode']):
    caseMaf = analyzeMaf[analyzeMaf['Tumor_Sample_Barcode'] == case]
    nOncMuts = caseMaf[caseMaf['oncogenic'].notnull()].shape[0]
    nOncGenes = len(set(caseMaf[caseMaf['oncogenic'].notnull()]['Hugo_Symbol']))
    #nOncMuts = len(set(caseMaf[caseMaf['oncogenic'].notnull()]['Hugo_Symbol']))
    nmuts = caseMaf.shape[0]
    nDoubleGenes = len(set(caseMaf[~(caseMaf['Hugo_Symbol'].isin(impactGenes)) & (caseMaf['isDouble'] == True)]['Hugo_Symbol']))
    
    listOfDicts.append({'Nmut': nmuts, 'Nonc': nOncMuts, 'NoncGenes': nOncGenes, 'oncRate': (1.0*nOncMuts)/nmuts,
                       'nDoubleTruncatingMutatedGenes': nDoubleGenes, 'truncDoubleRate': (1.0*nDoubleGenes)/nmuts,
                       'Tumor_Sample_Barcode': case[:15]})
    
df = pd.DataFrame(listOfDicts)


In [ ]:
df['aetiology'] = df['Tumor_Sample_Barcode'].apply(lambda x: 'POLE' if x in poleCases else 'MMR' if x in mmrCases else 'Other')

In [ ]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/passengerRates.tsv', index=False, sep='\t')

In [15]:
#
##
#####
reload(get_gene_and_cohort_list_utils)
essentialGenesInfo = get_gene_and_cohort_list_utils.get_essential_genes(depMapPath = pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/achillesDepMap.csv', mode='getAllGenes')


In [104]:
def summarize_mut_stats_for_genes_based_on_essentiality_category(maf, geneInfoDict, geneSizeDict):
    #manually setting breaks for this thing
    nCases = len(set(maf['Tumor_Sample_Barcode']))
    maf = maf[(maf['Hugo_Symbol'].isin(geneInfoDict.keys()))
              & (~maf['Hugo_Symbol'].isin(impactGenes))] #LEAVE ALL IMPACT GENES OUT OF THIS ANALYSIS
    #truncatingMaf = maf[maf['Consequence'].isin(set(['frameshift_variant', 'stop_gained']))]
    
    #TEMP CHANGE
    truncatingMaf = maf[maf['Consequence'].isin(set(['synonymous_variant']))]
    
    truncatingDoubleMaf = truncatingMaf[truncatingMaf['isDouble'] == True]
    listOfDicts = []
    cntr = 0
    for gene in set(maf['Hugo_Symbol']):
        cntr += 1
        if cntr%500==0:print cntr,
            
        if gene in geneSizeDict:
            geneSize = geneSizeDict[gene]
            geneMafTruncating = truncatingMaf[truncatingMaf['Hugo_Symbol'] == gene]
            countTruncating = len(set(geneMafTruncating['SAMPLE_ID']))
            doubletruncatingGeneMaf = truncatingDoubleMaf[truncatingDoubleMaf['Hugo_Symbol'] == gene]
            countDoubleTruncating = len(set(doubletruncatingGeneMaf['SAMPLE_ID']))
            listOfDicts.append({'gene': gene,  'score': geneInfoDict[gene],
                                'countTruncating': countTruncating, 'truncatingRate': (1.0*countTruncating)/(nCases*geneSize),
                                'countTruncatingDouble': countDoubleTruncating, 'truncatingDoubleRate': (1.0*countDoubleTruncating)/(nCases*geneSize)})
    df = pd.DataFrame(listOfDicts)
    return df
        

In [4]:
#get information about gene lengths in TCGA
geneLengthInfo = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/all_gene_coding_sequences.tsv')
geneLengthDict = dict(zip(geneLengthInfo['hgnc_symbol'], geneLengthInfo['nt.length']))

In [105]:
#MAKE ALL THE MAFS and Summaries

print 'analyzing pole maf'
poleMaf = analyzeMaf[analyzeMaf['SAMPLE_ID'].isin(poleCases)]
dfEssentialityPole = summarize_mut_stats_for_genes_based_on_essentiality_category(
        poleMaf, 
        dict(zip(essentialGenesInfo['Hugo_Symbol'], essentialGenesInfo['score'])),
        geneLengthDict)

print 'analyzing mmr maf'
mmrMaf = analyzeMaf[analyzeMaf['SAMPLE_ID'].isin(mmrCasesReduced)]
dfEssentialityMMR = summarize_mut_stats_for_genes_based_on_essentiality_category(
        mmrMaf, 
        dict(zip(essentialGenesInfo['Hugo_Symbol'], essentialGenesInfo['score'])),
        geneLengthDict)

print 'analyzing low mut burden maf'
lowMutBurdenMaf = analyzeMaf[analyzeMaf['SAMPLE_ID'].isin(lowTMBCases)]
dfEssentialityLowTMB = summarize_mut_stats_for_genes_based_on_essentiality_category(
        lowMutBurdenMaf, 
        dict(zip(essentialGenesInfo['Hugo_Symbol'], essentialGenesInfo['score'])),
        geneLengthDict)



analyzing pole maf
500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 11500 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500 17000 analyzing mmr maf
500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 11500 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500 17000 analyzing low mut burden maf
500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 11500 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500


In [101]:
dfEssentialityMMR['sigClass'] = 'MMR'
dfEssentialityPole['sigClass'] = 'POLE'
dfEssentialityLowTMB['sigClass'] = 'LOW-TMB'
dfEssentiality = pd.concat([dfEssentialityMMR, dfEssentialityPole, dfEssentialityLowTMB])
dfEssentiality['category'] = dfEssentiality['score'].apply(lambda x: '>0' if x > 0
                                                          else '-.5->0' if x > -.5
                                                          else '-1->-.5' if x > -1
                                                          else '<-1')



#dfEssentiality['category'] = dfEssentiality['score'].apply(lambda x: '>0' if x > 0
#                                                          else '-1->-0' if x > -1
#                                                          else '<-1')


dfEssentiality.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/doubleMissenseRatesByEssentiality.tsv', index=False, sep='\t')


In [ ]:

#
#######
####################
###########################
######################################

In [ ]:
#FOR DNDS

multipleMutationsMaf['sampleID'] = multipleMutationsMaf['SAMPLE_ID']
multipleMutationsMaf['chr'] = multipleMutationsMaf['Chromosome']
multipleMutationsMaf['pos'] = multipleMutationsMaf['Start_Position']
multipleMutationsMaf['ref'] = multipleMutationsMaf['Reference_Allele']
multipleMutationsMaf['mut'] = multipleMutationsMaf['Tumor_Seq_Allele2']

#dont include impact genes in essential genes
essentialGenes = essentialGenes - impactGenes

mMutationsMafEssentialGenes = multipleMutationsMaf[multipleMutationsMaf['Hugo_Symbol'].isin(essentialGenes)]
mMutationsMafTSGs = multipleMutationsMaf[multipleMutationsMaf['Hugo_Symbol'].isin(tsgs)]
mMutationsMafNonImpactNonEssential = multipleMutationsMaf[~multipleMutationsMaf['Hugo_Symbol'].isin(impactAndEssentialGenes)]

#make three different dndsMafs
dndsMafEssentialGenes = mMutationsMafEssentialGenes[['sampleID', 'chr', 'pos', 'ref', 'mut']]
dndsMafTSGs = mMutationsMafTSGs[['sampleID', 'chr', 'pos', 'ref', 'mut']]
dndsMafNonImpactNonEssential = mMutationsMafNonImpactNonEssential[['sampleID', 'chr', 'pos', 'ref', 'mut']]

dndsMafEssentialGenes.to_csv('/Users/friedman/Desktop/dndsMafEssentialTemp.tsv',index=False, sep='\t')
dndsMafTSGs.to_csv('/Users/friedman/Desktop/dndsMafTSGTemp.tsv',index=False, sep='\t')
dndsMafNonImpactNonEssential.to_csv('/Users/friedman/Desktop/dndsMafNonImpactNonEssentialTemp.tsv',index=False, sep='\t')

multipleMutationsMaf = multipleMutationsMaf[multipleMutationsMaf['sampleID'].isin(mmrCases)]
multipleMutationsMaf = multipleMutationsMaf[['sampleID', 'chr', 'pos', 'ref', 'mut']]

multipleMutationsMaf.to_csv('/Users/friedman/Desktop/dndsMafAllMMR.tsv', index=False, sep='\t')

In [ ]:
#
##
######
##########
#DO the per cases dnds cv stuff
mafDirectory = '/Users/friedman/Desktop/tempTCGAMafsForDNDSCV/'
for case in cases:
    caseMaf = multipleMutationsMaf[multipleMutationsMaf['sampleID'] == case]
    

In [ ]:
#
####
######
##########
######
####
#


In [ ]:
def get_n_non_synom_muts(maf):
    nonSynomConsequences = set(['missense_variant', 'frameshift_variant', 'stop_gained'])
    nonSynomMaf = maf[maf['Consequence'].isin(nonSynomConsequences)]
    return dict(nonSynomMaf['SAMPLE_ID'].value_counts())
    

In [ ]:
mmrCases = set(['TCGA-BR-8361-01', 'TCGA-D7-A4YV-01', 'TCGA-AA-3984-01', 'TCGA-D5-6530-01', 'TCGA-BR-6452-01', 'TCGA-VS-A94Z-01', 'TCGA-F1-6874-01', 'TCGA-CG-4306-01', 'TCGA-A6-2686-01', 'TCGA-AA-3492-01', 'TCGA-AJ-A3OL-01', 'TCGA-AX-A1C9-01', 'TCGA-HU-A4G8-01', 'TCGA-EJ-7782-01', 'TCGA-AP-A0LS-01', 'TCGA-HF-A5NB-01', 'TCGA-EC-A24G-01', 'TCGA-KN-8428-01', 'TCGA-EE-A29L-06', 'TCGA-AA-3713-01', 'TCGA-BR-4370-01', 'TCGA-D1-A177-01', 'TCGA-D1-A167-01', 'TCGA-L5-A43J-01', 'TCGA-B0-5098-01', 'TCGA-AD-6895-01', 'TCGA-CN-5369-01', 'TCGA-B5-A11G-01', 'TCGA-A6-6653-01', 'TCGA-EY-A1GK-01', 'TCGA-CM-4746-01', 'TCGA-DF-A2KZ-01', 'TCGA-SJ-A6ZI-01', 'TCGA-EA-A3HS-01', 'TCGA-HF-7132-01', 'TCGA-A5-A2K3-01', 'TCGA-D1-A17Q-01', 'TCGA-BR-4368-01', 'TCGA-D3-A8GM-06', 'TCGA-EE-A3AB-06', 'TCGA-BS-A0TA-01', 'TCGA-CD-A4MG-01', 'TCGA-EY-A5W2-01', 'TCGA-CK-5916-01', 'TCGA-BR-4201-01', 'TCGA-BR-7703-01', 'TCGA-D1-A101-01', 'TCGA-G4-6628-01', 'TCGA-G4-6309-01', 'TCGA-B5-A0K9-01', 'TCGA-AP-A1E1-01', 'TCGA-AO-A128-01', 'TCGA-QG-A5Z2-01', 'TCGA-BS-A0V4-01', 'TCGA-EW-A2FV-01', 'TCGA-VQ-AA6D-01', 'TCGA-D5-6540-01', 'TCGA-A5-A0VQ-01', 'TCGA-HU-8602-01', 'TCGA-D1-A1O7-01', 'TCGA-KK-A59V-01', 'TCGA-BR-4292-01', 'TCGA-MA-AA42-01', 'TCGA-A5-A2K7-01', 'TCGA-AD-6889-01', 'TCGA-AD-5900-01', 'TCGA-EE-A2GI-06', 'TCGA-BG-A0VW-01', 'TCGA-B5-A3FC-01', 'TCGA-AY-6197-01', 'TCGA-FI-A2F4-01', 'TCGA-EE-A3AA-06', 'TCGA-CD-8536-01', 'TCGA-D7-A4YY-01', 'TCGA-AD-A5EJ-01', 'TCGA-HU-A4H8-01', 'TCGA-BS-A0UV-01', 'TCGA-EY-A548-01', 'TCGA-AX-A2HJ-01', 'TCGA-NH-A5IV-01', 'TCGA-CM-5861-01', 'TCGA-AP-A1DO-01', 'TCGA-BG-A222-01', 'TCGA-D8-A1XK-01', 'TCGA-F7-A624-01', 'TCGA-AM-5821-01', 'TCGA-AA-A01R-01', 'TCGA-F4-6570-01', 'TCGA-CM-6171-01', 'TCGA-HU-A4H3-01', 'TCGA-G4-6588-01', 'TCGA-EY-A215-01', 'TCGA-CK-6746-01', 'TCGA-AX-A3FS-01', 'TCGA-E6-A2P8-01', 'TCGA-B7-5816-01', 'TCGA-EY-A1GF-01', 'TCGA-EE-A181-06', 'TCGA-BG-A18B-01', 'TCGA-BR-8363-01', 'TCGA-EO-A3KX-01', 'TCGA-BS-A0VI-01', 'TCGA-BS-A0UA-01', 'TCGA-EY-A2OM-01', 'TCGA-B5-A11E-01', 'TCGA-BG-A3EW-01', 'TCGA-VQ-A8E3-01', 'TCGA-BR-8368-01', 'TCGA-EY-A1GU-01', 'TCGA-AJ-A8CW-01', 'TCGA-F1-6177-01', 'TCGA-D5-6927-01', 'TCGA-EO-A22T-01', 'TCGA-AJ-A23O-01', 'TCGA-G4-6304-01', 'TCGA-BG-A2L7-01', 'TCGA-HU-A4H4-01', 'TCGA-MX-A5UJ-01', 'TCGA-VQ-A8PP-01', 'TCGA-A6-5665-01', 'TCGA-EY-A1GQ-01', 'TCGA-EC-A1NJ-01', 'TCGA-DI-A0WH-01', 'TCGA-D1-A103-01', 'TCGA-BR-8372-01', 'TCGA-BR-6566-01', 'TCGA-BR-8059-01', 'TCGA-D1-A1NS-01', 'TCGA-G4-6586-01', 'TCGA-4R-AA8I-01', 'TCGA-AP-A1DR-01', 'TCGA-D8-A1XQ-01', 'TCGA-B5-A11J-01', 'TCGA-BS-A0UJ-01', 'TCGA-EC-A1QX-01', 'TCGA-CG-4442-01', 'TCGA-AX-A1C5-01', 'TCGA-AP-A0LP-01', 'TCGA-B5-A5OC-01', 'TCGA-HU-A4G9-01', 'TCGA-CG-5721-01', 'TCGA-AX-A2HG-01', 'TCGA-VQ-A8P2-01', 'TCGA-BS-A0UL-01', 'TCGA-FI-A2D6-01', 'TCGA-95-7567-01', 'TCGA-AX-A3G9-01', 'TCGA-EA-A410-01', 'TCGA-AA-3510-01', 'TCGA-CV-A6K1-01', 'TCGA-63-A5MM-01', 'TCGA-B5-A1MX-01', 'TCGA-B5-A11Y-01', 'TCGA-BR-8487-01', 'TCGA-AX-A1C4-01', 'TCGA-D1-A17U-01', 'TCGA-D1-A17R-01', 'TCGA-HU-A4GN-01', 'TCGA-94-8490-01', 'TCGA-AA-A010-01', 'TCGA-EE-A29E-06', 'TCGA-F4-6856-01', 'TCGA-D1-A2G0-01', 'TCGA-EI-6882-01', 'TCGA-BR-8081-01', 'TCGA-CD-A4MJ-01', 'TCGA-EI-6507-01', 'TCGA-BR-4256-01', 'TCGA-AZ-4315-01', 'TCGA-CM-6674-01', 'TCGA-CM-4743-01', 'TCGA-PG-A917-01', 'TCGA-AJ-A2QO-01', 'TCGA-DF-A2KN-01', 'TCGA-N7-A4Y0-01', 'TCGA-CA-6717-01', 'TCGA-D5-6930-01', 'TCGA-IB-7651-01', 'TCGA-AA-A01P-01', 'TCGA-F4-6703-01', 'TCGA-EO-A22R-01', 'TCGA-DM-A1HB-01', 'TCGA-AX-A1CE-01', 'TCGA-A5-A0G1-01', 'TCGA-EO-A22S-01', 'TCGA-BR-7851-01', 'TCGA-AA-3663-01', 'TCGA-D1-A0ZS-01', 'TCGA-B5-A0JY-01', 'TCGA-2W-A8YY-01', 'TCGA-EO-A3AZ-01', 'TCGA-B5-A11H-01', 'TCGA-AX-A2HD-01', 'TCGA-D1-A17D-01', 'TCGA-BR-7707-01', 'TCGA-BR-8680-01', 'TCGA-PK-A5HB-01', 'TCGA-AX-A0J0-01', 'TCGA-AX-A3FT-01', 'TCGA-AJ-A3BH-01', 'TCGA-VS-A9U6-01', 'TCGA-BR-4362-01', 'TCGA-A6-5661-01', 'TCGA-SL-A6JA-01', 'TCGA-D1-A1NY-01', 'TCGA-CK-4951-01', 'TCGA-L5-A4OI-01', 'TCGA-QS-A5YR-01', 'TCGA-BR-8078-01', 'TCGA-EY-A1H0-01', 'TCGA-QK-A6VB-01', 'TCGA-AA-3947-01', 'TCGA-B7-A5TI-01', 'TCGA-VS-A9UJ-01', 'TCGA-VQ-A91K-01', 'TCGA-AP-A1DV-01', 'TCGA-AP-A059-01', 'TCGA-EO-A3AU-01', 'TCGA-B5-A11U-01', 'TCGA-BR-4363-01', 'TCGA-F5-6814-01', 'TCGA-G4-6302-01', 'TCGA-BS-A0TJ-01', 'TCGA-ZF-AA4W-01', 'TCGA-FI-A2D4-01', 'TCGA-DU-6392-01', 'TCGA-BS-A0TE-01', 'TCGA-AP-A1DK-01', 'TCGA-D8-A1J8-01', 'TCGA-EO-A22U-01', 'TCGA-VQ-A8PB-01', 'TCGA-VQ-A8PX-01', 'TCGA-A5-A0VP-01', 'TCGA-D1-A15X-01', 'TCGA-EE-A2MD-06', 'TCGA-AP-A0LM-01', 'TCGA-BK-A13B-01', 'TCGA-CG-5723-01', 'TCGA-FI-A2D0-01', 'TCGA-BR-6802-01', 'TCGA-CG-5726-01', 'TCGA-CG-4465-01', 'TCGA-DK-A6AW-01', 'TCGA-AJ-A3BG-01', 'TCGA-D5-6928-01', 'TCGA-EW-A1IZ-01', 'TCGA-CD-A4MI-01', 'TCGA-BR-8382-01', 'TCGA-D1-A174-01', 'TCGA-AJ-A3EK-01', 'TCGA-D1-A17M-01', 'TCGA-WQ-A9G7-01', 'TCGA-D1-A0ZO-01', 'TCGA-BR-6852-01', 'TCGA-AP-A1DM-01', 'TCGA-EI-6917-01', 'TCGA-D1-A17H-01', 'TCGA-SW-A7EA-01', 'TCGA-BH-A18G-01', 'TCGA-A6-A565-01', 'TCGA-D1-A1NZ-01', 'TCGA-AU-6004-01', 'TCGA-VQ-A924-01', 'TCGA-DI-A1BU-01', 'TCGA-CK-5913-01', 'TCGA-R5-A7ZI-01', 'TCGA-BR-4257-01', 'TCGA-AP-A0LE-01', 'TCGA-06-5416-01', 'TCGA-D1-A160-01', 'TCGA-E6-A1LX-01', 'TCGA-AZ-4615-01', 'TCGA-FI-A2D5-01', 'TCGA-EE-A2GO-06', 'TCGA-AZ-6598-01', 'TCGA-B5-A0JU-01', 'TCGA-AJ-A3EL-01', 'TCGA-WS-AB45-01', 'TCGA-A5-A1OF-01', 'TCGA-AX-A06H-01', 'TCGA-BS-A0U8-01', 'TCGA-55-8506-01', 'TCGA-AP-A1DP-01', 'TCGA-F1-A448-01', 'TCGA-HU-A4GX-01', 'TCGA-AX-A3G8-01', 'TCGA-BR-8591-01', 'TCGA-BG-A0MQ-01', 'TCGA-B5-A1MW-01', 'TCGA-BR-4280-01', 'TCGA-AX-A2IN-01', 'TCGA-AD-6964-01', 'TCGA-D1-A163-01', 'TCGA-D7-A6EY-01', 'TCGA-VQ-A91D-01', 'TCGA-E6-A2P9-01', 'TCGA-QQ-A8VF-01', 'TCGA-HU-A4GU-01', 'TCGA-EO-A22X-01', 'TCGA-PG-A6IB-01', 'TCGA-ZB-A966-01', 'TCGA-CG-4305-01', 'TCGA-RD-A8NB-01', 'TCGA-BH-A0HA-01', 'TCGA-86-A4JF-01', 'TCGA-B5-A3FA-01', 'TCGA-XK-AAIW-01', 'TCGA-DF-A2KU-01', 'TCGA-VQ-A8PO-01', 'TCGA-BR-4361-01', 'TCGA-AP-A1DH-01', 'TCGA-EY-A2OP-01', 'TCGA-AJ-A3OJ-01', 'TCGA-G4-6320-01', 'TCGA-5M-AAT6-01', 'TCGA-BG-A0VZ-01', 'TCGA-HU-A4GT-01', 'TCGA-ND-A4WC-01', 'TCGA-D1-A17A-01', 'TCGA-3B-A9HT-01', 'TCGA-EY-A1GO-01', 'TCGA-EO-A3B0-01', 'TCGA-HU-A4GQ-01', 'TCGA-L5-A8NM-01', 'TCGA-D1-A176-01', 'TCGA-AX-A2HA-01'])
poleCases = set(['TCGA-DU-6392-01', 'TCGA-AA-3984-01', 'TCGA-AJ-A3EK-01', 'TCGA-B5-A0JY-01', 'TCGA-AX-A2HD-01', 'TCGA-AP-A1E0-01', 'TCGA-AZ-4315-01', 'TCGA-EO-A3AV-01', 'TCGA-DF-A2KU-01', 'TCGA-E6-A1M0-01', 'TCGA-EI-6917-01', 'TCGA-DF-A2KN-01', 'TCGA-AX-A2HC-01', 'TCGA-EO-A3AY-01', 'TCGA-QF-A5YS-01', 'TCGA-B5-A11N-01', 'TCGA-EO-A3KX-01', 'TCGA-B5-A11E-01', 'TCGA-AX-A1CE-01', 'TCGA-BS-A0UF-01', 'TCGA-ND-A4WC-01', 'TCGA-06-5416-01', 'TCGA-EO-A22R-01', 'TCGA-2W-A8YY-01', 'TCGA-FI-A2D5-01', 'TCGA-AJ-A3EL-01', 'TCGA-B5-A3FA-01', 'TCGA-AJ-A5DW-01', 'TCGA-CA-6717-01', 'TCGA-XK-AAIW-01', 'TCGA-BR-8680-01', 'TCGA-AX-A0J0-01', 'TCGA-D1-A103-01', 'TCGA-AJ-A3BH-01', 'TCGA-BK-A6W3-01', 'TCGA-D1-A16X-01', 'TCGA-IB-7651-01', 'TCGA-E6-A1LX-01', 'TCGA-A5-A1OF-01', 'TCGA-BS-A0TC-01', 'TCGA-EW-A2FV-01', 'TCGA-EO-A22X-01', 'TCGA-D1-A17Q-01', 'TCGA-AP-A1DV-01', 'TCGA-AP-A059-01', 'TCGA-A5-A0G1-01', 'TCGA-VQ-A8P2-01', 'TCGA-EO-A3B0-01', 'TCGA-D1-A16Y-01', 'TCGA-F5-6814-01', 'TCGA-EY-A1GI-01', 'TCGA-AA-3510-01', 'TCGA-B5-A3FC-01', 'TCGA-AP-A1DK-01', 'TCGA-DK-A6AW-01', 'TCGA-EO-A22U-01', 'TCGA-QS-A5YQ-01', 'TCGA-B5-A1MR-01', 'TCGA-A5-A2K5-01', 'TCGA-BS-A0UV-01', 'TCGA-AP-A0LM-01', 'TCGA-DF-A2KV-01', 'TCGA-AA-A010-01', 'TCGA-EY-A215-01', 'TCGA-B5-A1MX-01', 'TCGA-CA-6718-01', 'TCGA-EY-A1GD-01'])
hypermutantIds = mmrCases | poleCases
hypermutantMaf = mc3maf[mc3maf['SAMPLE_ID'].isin(hypermutantIds)]


In [ ]:
nonSynomDict = get_n_non_synom_muts(hypermutantMaf)
nonsenseConsequences = set(['frameshift_variant', 'stop_gained'])
nonsenseMuts = hypermutantMaf[hypermutantMaf['Consequence'].isin(nonsenseConsequences)]

In [ ]:
listOfDicts = []
for case in set(nonsenseMuts['SAMPLE_ID']):
    caseMuts = nonsenseMuts[nonsenseMuts['SAMPLE_ID'] == case]
    geneCounts = caseMuts['Hugo_Symbol'].value_counts()
    
    doubleGenes = set([entry[0] for entry in geneCounts.items() if entry[1] > 1])
    doubleMuts = caseMuts[caseMuts['Hugo_Symbol'].isin(doubleGenes)]
    doubleMutsEssential = doubleMuts[doubleMuts['Hugo_Symbol'].isin(essentialGenes)]
    nEssentialDouble = doubleMutsEssential.shape[0]
    nEssential = caseMuts[caseMuts['Hugo_Symbol'].isin(essentialGenes)].shape[0]
    
    nNonSynom = nonSynomDict[case]
    
    listOfDicts.append({'Tumor_Sample_Barcode': case, 'nEssential': nEssential, 'nDouble': len(doubleGenes),
                        'nEssentialDouble': nEssentialDouble, 'nNonSynom': nNonSynom})
    
df = pd.DataFrame(listOfDicts)
    #caseMutsEssentialGenes = caseMuts[caseMuts['Hugo_Symbol'].isin(essentialGenes)]
    #print caseMutsEssentialGenes.shape

In [ ]:
geneLengthInfo = pd.read_table(pathPrefix + '/ifs/res/taylorlab/ang46/ext/resources/coding_sequences_allgenes.tsv')
geneLengthDict = dict(zip(geneLengthInfo['hgnc_symbol'], geneLengthInfo['nt.length']))

In [ ]:
nEssentialGeneBases = sum(geneLengthInfo[geneLengthInfo['hgnc_symbol'].isin(essentialGenes)]['nt.length'])
nNeutralGeneBases = sum(geneLengthInfo[~geneLengthInfo['hgnc_symbol'].isin(essentialGenes | impactGenes)]['nt.length'])
nTSGGeneBases = sum(geneLengthInfo[geneLengthInfo['hgnc_symbol'].isin(tsgs)]['nt.length'])


In [ ]:
#hypermutantMaf2 = hypermutantMaf[hypermutantMaf['n_depth'] > 25]
nonSynomDict = get_n_non_synom_muts(hypermutantMaf)

hypermutantMaf2 = hypermutantMaf
listOfDicts = []
nonSynomConsequences = set(['missense_variant', 'frameshift_variant', 'stop_gained'])
cntr = 0
for case in set(hypermutantMaf2['SAMPLE_ID']):

    cntr += 1
    if cntr%50==0: print cntr
    
    caseMuts = hypermutantMaf2[hypermutantMaf2['SAMPLE_ID'] == case]
    geneCounts = caseMuts['Hugo_Symbol'].value_counts()
    doubleGenes = set([entry[0] for entry in geneCounts.items() if entry[1] > 1])
    
    doubleGeneCasesMuts = caseMuts[caseMuts['Hugo_Symbol'].isin(doubleGenes)]
    nonsenseConsequences = set(['frameshift_variant', 'stop_gained'])
    truncatingMuts = doubleGeneCasesMuts[doubleGeneCasesMuts['Consequence'].isin(nonsenseConsequences)]
    genesWithTwoMutsOneOfWhichIsTruncating = set(doubleGeneCasesMuts[doubleGeneCasesMuts['Consequence'].isin(nonsenseConsequences)]['Hugo_Symbol'])
    
    doubleWithOneTruncEssentialGenes = set(doubleGeneCasesMuts[(doubleGeneCasesMuts['Hugo_Symbol'].isin(essentialGenes)) &
                                                      (doubleGeneCasesMuts['Hugo_Symbol'].isin(genesWithTwoMutsOneOfWhichIsTruncating))]['Hugo_Symbol'])
    doubleWithOneTruncNeutralGenes = set(doubleGeneCasesMuts[(~doubleGeneCasesMuts['Hugo_Symbol'].isin(essentialGenes | impactGenes)) &
                                                      (doubleGeneCasesMuts['Hugo_Symbol'].isin(genesWithTwoMutsOneOfWhichIsTruncating))]['Hugo_Symbol'])
    doubleWithOneTruncTSGGenes = set(doubleGeneCasesMuts[(doubleGeneCasesMuts['Hugo_Symbol'].isin(tsgs)) &
                                                      (doubleGeneCasesMuts['Hugo_Symbol'].isin(genesWithTwoMutsOneOfWhichIsTruncating))]['Hugo_Symbol'])
    
    essentialTruncatingMuts = truncatingMuts[truncatingMuts['Hugo_Symbol'].isin(doubleWithOneTruncEssentialGenes)]
    neutralTruncatingMuts = truncatingMuts[truncatingMuts['Hugo_Symbol'].isin(doubleWithOneTruncNeutralGenes)]
    tsgTruncatingMuts = truncatingMuts[truncatingMuts['Hugo_Symbol'].isin(doubleWithOneTruncTSGGenes)]
    
    nDoubleEssentialTruncating = len(set([entry[0] for entry in essentialTruncatingMuts['Hugo_Symbol'].value_counts().items() if entry[1] > 1]))
    nDoubleNeutralTruncating = len(set([entry[0] for entry in neutralTruncatingMuts['Hugo_Symbol'].value_counts().items() if entry[1] > 1]))
    nDoubleTSGTruncating = len(set([entry[0] for entry in tsgTruncatingMuts['Hugo_Symbol'].value_counts().items() if entry[1] > 1]))
    
    fracEssentialDouble = None
    if len(doubleWithOneTruncEssentialGenes) > 0:
        fracEssentialDouble = (1.0*nDoubleEssentialTruncating)/len(doubleWithOneTruncEssentialGenes)
    fracNeutralDouble = None
    if len(doubleWithOneTruncNeutralGenes) > 0:
        fracNeutralDouble = (1.0*nDoubleNeutralTruncating)/len(doubleWithOneTruncNeutralGenes)
    fracTSGDouble = None
    if len(doubleWithOneTruncTSGGenes) > 0:
        fracTSGDouble = (1.0*nDoubleTSGTruncating)/len(doubleWithOneTruncTSGGenes)
    
    
    nmut = nonSynomDict[case]
    listOfDicts.append({'fracEssentialDouble': fracEssentialDouble, 'fracNeutralDouble': fracNeutralDouble,'fracTSGDouble': fracTSGDouble,
                        'nmut': nmut,
                        'nEssentialDouble': nDoubleEssentialTruncating, 'nNeutralDouble': nDoubleNeutralTruncating,'nTSGDouble': nDoubleTSGTruncating,
                       'nEssentialDoubleNormed': (1.0*nDoubleEssentialTruncating)/nEssentialGeneBases, 'nNeutralDoubleNormed': (1.0*nDoubleNeutralTruncating)/nNeutralGeneBases,'nTSGDoubleNormed': (1.0*nDoubleTSGTruncating)/nTSGGeneBases})
    
df = pd.DataFrame(listOfDicts)

In [ ]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/essentialGenesMuts.tsv', index=False, sep='\t')